In [ ]:
cd 

'/content'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/NLUProject

/content/drive/MyDrive/NLUProject


In [3]:
!pip install datasets

import re
from datasets import load_dataset
import spacy
from spacy.util import minibatch
import pandas as pd
import random

     |████████████████████████████████| 325 kB 12.6 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 136 kB 40.8 MB/s 
     |████████████████████████████████| 212 kB 48.5 MB/s 
     |████████████████████████████████| 1.1 MB 46.0 MB/s 
     |████████████████████████████████| 127 kB 29.2 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
     |████████████████████████████████| 144 kB 53.2 MB/s 
     |████████████████████████████████| 271 kB 55.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
#Get male words list
male = open("Data/GenderWordData/male_word_file.txt", "r")
male_data = male.read()
male_list = male_data.replace('\n', ' ').split(" ")
male.close()

#Get female words list
female = open("Data/GenderWordData/female_word_file.txt", "r")
female_data = female.read()
female_list = female_data.replace('\n', ' ').split(" ")
female.close()

In [ ]:
#Function to classify overall gender majority in text

def classify_text(text_lst,male_list, female_list):
    #get rid of punctuation
    results = []
    for i in range(0,len(text_lst)):
        text = re.sub(r'[^\w\s]', '', text_lst[i])    
        words = text.split(' ')
        male_count = 0
        female_count = 0
        for word in words:
            word_filter = word.strip().lower()
            if word_filter != '':
                if word_filter in male_list:
                    male_count = male_count + 1
                elif word_filter in female_list:
                    female_count = female_count + 1
        if female_count > male_count:
            results.append(1)
        elif male_count > female_count:
            results.append(0)
        else:
            results.append(-1)
    return results

In [ ]:
#Load Dataset
dataset = load_dataset("sentiment140")
train = dataset["train"]
test = dataset["test"]

Generating train split:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/498 [00:00<?, ? examples/s]

Dataset sentiment140 downloaded and prepared to /root/.cache/huggingface/datasets/sentiment140/sentiment140/1.0.0/f81c014152931b776735658d8ae493b181927de002e706c4d5244ecb26376997. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#Label Gender
train_classified = classify_text(train["text"],male_list,female_list)
test_classified = classify_text(train["text"],male_list,female_list)

In [ ]:
#Classify train set
train_df = pd.DataFrame(list(zip(train["text"],train["sentiment"],train_classified)),
                    columns = ["Text","Label", "Gender"])
train_df

,Text,Label,Gender
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,-1
1,is upset that he can't update his Facebook by ...,0,0
2,@Kenichan I dived many times for the ball. Man...,0,-1
3,my whole body feels itchy and like its on fire,0,-1
4,"@nationwideclass no, it's not behaving at all....",0,-1
...,...,...,...
1599995,Just woke up. Having no school is the best fee...,4,-1
1599996,TheWDB.com - Very cool to hear old Walt interv...,4,-1
1599997,Are you ready for your MoJo Makeover? Ask me f...,4,-1
1599998,Happy 38th Birthday to my boo of alll time!!! ...,4,-1


In [ ]:
#Classify test set
test_df = pd.DataFrame(list(zip(test["text"],test["sentiment"],test_classified)),
                    columns = ["Text","Label", "Gender"])
test_df

,Text,Label,Gender
0,@stellargirl I loooooooovvvvvveee my Kindle2. ...,4,-1
1,Reading my kindle2... Love it... Lee childs i...,4,0
2,"Ok, first assesment of the #kindle2 ...it fuck...",4,-1
3,@kenburbary You'll love your Kindle2. I've had...,4,-1
4,@mikefish Fair enough. But i have the Kindle2...,4,-1
...,...,...,...
493,Ask Programming: LaTeX or InDesign?: submitted...,2,-1
494,"On that note, I hate Word. I hate Pages. I hat...",0,-1
495,Ahhh... back in a *real* text editing environm...,4,-1
496,"Trouble in Iran, I see. Hmm. Iran. Iran so far...",0,-1


In [ ]:
#Combine datasets
data = pd.concat([train_df,test_df])

#Get rid of ungendered senetences 
data = data[data.Gender !=-1]

#Get rid of neutral sentiments
data = data[data.Label != 2]

#Use 0-1 sentiment labbeling
data['Label'] = data['Label'].replace([4],1)

data = data.reset_index()


In [ ]:
#Load Toxicity Models
toxicClassifierModelSpacy =  spacy.load("Transformer/Models/ToxicTextClassifierModel/model")
toxicSpanModelSpacy = spacy.load("Transformer/Models/ToxicTextSpanDetectionModel/model")


In [ ]:
#Function to mask toxic elements
def removeToxicity(text,model,mask):
    doc = model(text)
    if len(doc.ents) > 0:
        for word in doc.ents[0]:
            text = text.replace(word.text, mask)
    return text


In [ ]:
data["Text"] = data["Text"].apply(removeToxicity, args = (toxicSpanModelSpacy,"[TOXIC]"))

In [ ]:
#save?

In [ ]:
data

,index,Text,Label,Gender
0,1,is upset that he can't update his Facebook by ...,0,0
1,15,@iamjazzyfizzle I wish I got to watch it with ...,0,1
2,21,"one of my friend called me, and asked to meet ...",0,1
3,33,@julieebaby awe i love you too!!!! 1 am here ...,0,1
4,39,Bed. Class 8-12. Work 12-3. Gym 3-5 or 6. Then...,0,1
...,...,...,...,...
226918,453,I can't watch TV without a Tivo. And after al...,0,1
226919,461,Fuzzball is more fun than AT&amp;T ;P http://f...,0,1
226920,480,Fighting with LaTex. Again...,0,0
226921,481,@Iheartseverus we love you too and don't want ...,0,1


In [ ]:
data.to_pickle('data.pkl')

NameError: ignored

In [ ]:
data

NameError: ignored

In [4]:
cd drive/MyDrive/NLUProject

[Errno 2] No such file or directory: 'drive/MyDrive/NLUProject'
/content/drive/MyDrive/NLUProject


In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 4.0 MB 12.3 MB/s 
     |████████████████████████████████| 813 kB 42.1 MB/s 
     |████████████████████████████████| 253 kB 49.3 MB/s 
     |████████████████████████████████| 829 kB 35.1 MB/s 
     |████████████████████████████████| 636 kB 37.3 MB/s 
     |████████████████████████████████| 408 kB 51.1 MB/s 
     |████████████████████████████████| 895 kB 16.7 MB/s 
     |████████████████████████████████| 6.6 MB 28.4 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=025407e1c55a97f28598e53d64ebe25834436eb3a78a4ce96f0771875d5ecf13
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-

In [1]:
!pip install pytorch_lightning
import pytorch_lightning as pl


ImportError: ignored

In [6]:
from pl_classifier import NLU_dataset, NLU_DataModule, NLU_Toxic_classifier
import pickle
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
from pl_bolts.callbacks import PrintTableMetricsCallback
from sklearn.preprocessing import MultiLabelBinarizer


import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

import seaborn as sns
import matplotlib.pyplot as plt
import re
import pandas as pd

import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AdamW, BertConfig
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
from transformers import RobertaConfig, RobertaModel
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

import pandas as pd
import numpy as np
import random
import time
import datetime

#parameters
N_EPOCHS = 3
BATCH_SIZE = 12
MAX_LEN = 256
LR = 2e-05
opt_thresh = 0.4

directory = '/content/drive/MyDrive/NLUProject/'

# with open(directory+'data.pkl', 'rb') as f:
#     data = pickle.load(f)

data = pd.read_csv("/content/drive/MyDrive/NLUProject/Data/SentimentAnalysisData/Detox_Dataset")

mlb = MultiLabelBinarizer()
data['Label'] = mlb.fit_transform(data.Label.astype(str)).tolist()

steps_per_epoch= data.shape[0] // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

# new_model = NLU_Toxic_classifier.load_from_checkpoint(checkpoint_path=directory+"model_weights/hosrevroberta_210825_5.ckpt", n_classes=len(LABEL_COLUMNS))
# new_model.eval()

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
special_tokens_dict = {'additional_special_tokens': ['[TOXIC]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

#9:1 split
train_df, val_df = train_test_split(data, test_size=0.1)
data_module = NLU_DataModule(
  train_df,
  val_df,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_LEN)
data_module.setup()

#Creating Model 
new_model = NLU_Toxic_classifier(n_classes=2,
                              n_warmup_steps=warmup_steps,
                              n_training_steps=total_training_steps)
new_model.resize_token_embeddings(len(tokenizer))

trainer = pl.Trainer(max_epochs=N_EPOCHS, gpus=1, progress_bar_refresh_rate=3)

trainer.fit(new_model, data_module)

trainer.save_checkpoint("/content/drive/MyDrive/NLUProject/DeBERTa_220407_1.ckpt")


ImportError: ignored

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]])

In [ ]:
trainer.predict(data.iloc[0].Text)

AttributeError: ignored

In [ ]:
data.iloc[119].Label

0

In [ ]:

mlb = MultiLabelBinarizer()
data['Label'] = mlb.fit_transform(data.Label.astype(str)).tolist()
data

,index,Text,Label,Gender
0,1,is upset that he can't update his Facebook by ...,"[0, 1]",0
1,15,@iamjazzyfizzle I wish I got to watch it with ...,"[0, 1]",1
2,21,"one of my friend called me, and asked to meet ...","[0, 1]",1
3,33,@julieebaby awe i love you too!!!! 1 am here ...,"[0, 1]",1
4,39,Bed. Class 8-12. Work 12-3. Gym 3-5 or 6. Then...,"[0, 1]",1
...,...,...,...,...
226918,453,I can't watch TV without a Tivo. And after al...,"[0, 1]",1
226919,461,Fuzzball is more fun than AT&amp;T ;P http://f...,"[0, 1]",1
226920,480,Fighting with LaTex. Again...,"[0, 1]",0
226921,481,@Iheartseverus we love you too and don't want ...,"[0, 1]",1


In [ ]:
max_token_len=256
test_sibal = []
for i in data.index[:10]:
  data_row = data.iloc[i]

  texts = data_row.Text
  labels = data_row.Label

  encoding = tokenizer.encode_plus(texts,
                                        add_special_tokens=True,
                                        max_length=max_token_len,
                                        return_token_type_ids=False,
                                        padding="max_length",
                                        truncation=True,
                                        return_attention_mask=True,
                                        return_tensors='pt')
  test_sibal.append(dict(texts=texts,
        input_ids=encoding["input_ids"].flatten(),
        attention_mask=encoding["attention_mask"].flatten(),
        labels=torch.FloatTensor(labels)))

In [ ]:
pd.DataFrame(test_sibal)

,texts,input_ids,attention_mask,labels
0,is upset that he can't update his Facebook by ...,"[tensor(1), tensor(354), tensor(4904), tensor(...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",[tensor(3.1503e-11)]
1,@iamjazzyfizzle I wish I got to watch it with ...,"[tensor(1), tensor(1039), tensor(6009), tensor...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",[tensor(3.1508e-11)]
2,"one of my friend called me, and asked to meet ...","[tensor(1), tensor(1264), tensor(9), tensor(12...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",[tensor(3.1509e-11)]
3,@julieebaby awe i love you too!!!! 1 am here ...,"[tensor(1), tensor(1039), tensor(267), tensor(...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",[tensor(3.1509e-11)]
4,Bed. Class 8-12. Work 12-3. Gym 3-5 or 6. Then...,"[tensor(1), tensor(44391), tensor(4), tensor(4...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",[tensor(3.1510e-11)]
5,Falling asleep. Just heard about that Tracy gi...,"[tensor(1), tensor(597), tensor(9391), tensor(...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",[tensor(3.1510e-11)]
6,"@BatManYNG I miss my ps3, it's out of commissi...","[tensor(1), tensor(1039), tensor(43852), tenso...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",[tensor(3.1510e-11)]
7,@jdarter Oh! Haha... dude I dont really look a...,"[tensor(1), tensor(1039), tensor(267), tensor(...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",[tensor(3.1510e-11)]
8,"Gym attire today was: Puma singlet, Adidas sho...","[tensor(1), tensor(534), tensor(8307), tensor(...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",[tensor(3.1510e-11)]
9,@msdrama hey missed ya at the meeting sup mama,"[tensor(1), tensor(1039), tensor(4339), tensor...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",[tensor(3.1503e-11)]


In [ ]:
data_row.Label.tolist()

0

In [ ]:
torch.Tensor([1])

tensor([1.])

In [ ]:
1.3358e-08 == 1

False

[1, 2, 3]